## ARC 380 / CEE 380 / ROB 380 – Introduction to Robotics for Digital Fabrication  
### Session 6 Workshop: ROS Fundamentals
Princeton University, Spring 2026  
Professor: Arash Adel | TA: Daniel Ruan  

---

# 1. Introduction to ROS

**ROS (Robot Operating System)** is an open-source middleware and ecosystem for building robot software.  
It provides conventions, libraries, and tools that make it easier to build **distributed** robot systems composed of many small processes that communicate over well-defined interfaces.

Common reasons to use ROS:
- **Modularity**: break a robot application into reusable components (nodes).
- **Interoperability**: standardized message, service, and action interfaces across packages.
- **Tooling**: logging, introspection, debugging, visualization, and simulation tools (covered next week).
- **Ecosystem**: large community and many reusable packages.

This course uses **ROS 2 Jazzy** (EOL date May 2029).  

<img src="./images/JazzyJalisco.png" alt="ROS Jazzy Jalisco" width="200"/>

## 1.1. ROS 1 vs ROS 2 

ROS 2 was designed to address limitations of ROS 1 for modern robotics use cases (multi-robot, unreliable networks, real-time, security, embedded).

Key differences you should know:

### Architecture & transport
- **ROS 1**: centralized discovery via the **ROS master** (`roscore`). Transport typically uses TCPROS/UDPROS.
- **ROS 2**: decentralized discovery via **DDS** (Data Distribution Service). No single master required.

### Communication behavior
- **ROS 1**: limited built-in control over transport behavior.
- **ROS 2**: **QoS (Quality of Service)** controls reliability, durability, history depth, deadlines, etc.

### Build system & packages
- **ROS 1**: `catkin` + `rosbuild` history; common workspace style: `catkin_ws`.
- **ROS 2**: `ament` + `colcon`; common workspace style: `ros2_ws`.

### Real-time & embedded
- **ROS 2**: designed with real-time and embedded deployments in mind (executor/callback model, DDS implementations, lifecycle nodes).

### Security
- **ROS 1**: no standard security.
- **ROS 2**: DDS Security (SROS2) provides authentication, encryption, and access control (when configured).

In this workshop, we focus on the **ROS 2 computation graph** and the core communication primitives: **topics, services, actions**.  


## 1.2. The ROS 2 computation graph  

At runtime, a ROS 2 system is a **graph** of communicating processes:

- **Nodes**: individual processes (or components) that do work.
- **Topics**: named channels for streaming messages (publish/subscribe).
- **Services**: request/response interactions (client/server).
- **Actions**: long-running goals with feedback and cancelation (client/server).
- **Parameters**: runtime configuration for nodes.
- **Interfaces**: message (`.msg`), service (`.srv`), and action (`.action`) definitions.

You can think of this as “many small programs + well-defined APIs + tools to observe the graph”.  


# 2. Nodes

A **node** is the basic unit of execution in ROS 2. Nodes:
- expose publishers/subscribers, services/clients, and action servers/clients
- declare and use parameters
- run callbacks when data arrives or timers fire

In ROS 2 Python, nodes are written using **`rclpy`**.

## 2.1. Callback + executor model (high level)
- Your node registers **callbacks** (e.g., “when a message arrives”, “every 0.1s”).
- An **executor** spins and dispatches callbacks.
- For this workshop, you can treat `rclpy.spin(node)` as “run the node until shutdown”.

## 2.2 Minimal node skeleton (Python)
```python
import rclpy
from rclpy.node import Node

class MyNode(Node):
    def __init__(self):
        super().__init__('my_node_name')
        self.get_logger().info("Hello from ROS 2!")

def main():
    rclpy.init()
    node = MyNode()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()
```

Notes:
- Node names must be unique in a running graph (or use namespaces).
- `get_logger()` is the standard way to print structured logs.  


# 3. Topics (publish / subscribe)  

A **topic** is a named stream of typed messages. Topics are best for:
- sensor streams
- robot state updates
- commands that update frequently (or can be overwritten)

## 3.1. Core ideas
- **Publisher** sends messages of a specific type on a topic name.
- **Subscriber** receives messages of the same type from the same topic name.
- Topics are **many-to-many**: many publishers and many subscribers can share a topic.

<img src="./images/Topic-MultiplePublisherandMultipleSubscriber.gif" alt="ROS Topics" width="600"/>

## 3.2. Message types
- Message types are defined in `.msg` files and compiled into language-specific code.
- ROS provides many common message types (e.g., `std_msgs`, `geometry_msgs`, `sensor_msgs`).
- You can find the documentation for interfaces (including messages) online through the [ROS Index](https://index.ros.org/), for example for `std_msgs`: https://index.ros.org/p/std_msgs/#jazzy-assets

## 3.3. QoS (Quality of Service) — ROS 2-specific
QoS controls delivery behavior, parameterized by:
- **reliability**: reliable vs best-effort
- **durability**: volatile vs transient-local (late-joiners may get last message)
- **history/depth**: how many messages to buffer

For this workshop, we will use default QoS unless a task requires otherwise.  


# 4. Services (request / response)  

A **service** is a synchronous request/response API:
- **client** sends a request
- **server** computes and returns a response

<img src="./images/Service-SingleServiceClient.gif" alt="ROS Services" width="600"/>

Services are best for:
- “ask once, get one answer” interactions  
  (e.g., “reset”, “get status”, “compute something immediately”)

## 4.1. Service types
Service interfaces are defined in `.srv` files with:
- request fields
- response fields

Example ([example_interfaces/srv/AddTwoInts.srv](https://github.com/ros2/example_interfaces/blob/jazzy/srv/AddTwoInts.srv)):
```
int64 a
int64 b
---
int64 sum
```


# 5. Actions (goals, feedback, result, cancelation)  

An **action** is a goal-based API for tasks that take time:
- client sends a **goal**
- server may accept/reject
- server periodically publishes **feedback**
- server eventually returns a **result**
- client can **cancel** the goal

<img src="./images/Action-SingleActionClient.gif" alt="ROS Actions" width="600"/>

Actions are best for:
- navigation goals
- manipulation tasks
- any operation where progress matters and cancelation is useful

## 5.1. Action types
Action interfaces are defined in `.action` files with:
- goal fields
- result fields
- feedback fields

Example ([example_interfaces/action/Fibonacci.action](https://github.com/ros2/example_interfaces/blob/jazzy/action/Fibonacci.action)):
```
# Goal
int32 order
---
# Result
int32[] sequence
---
# Feedback
int32[] partial_sequence
```


# 6. Code-Along

We will implement the following using **ROS 2 Python (`rclpy`)**:

## 6.1. Example A — Talker / Listener (topics)
**Goal**: write two nodes that communicate over a topic.
- `talker.py`: publishes a message periodically (use a timer)
- `listener.py`: subscribes and prints received messages

What you should practice:
- creating publishers/subscribers
- message types (`std_msgs/msg/String` or similar)
- timers and callbacks
- logging

## 6.2. Example B — AddTwoInts server & client (services)
**Goal**: implement request/response computation.
- `add_two_ints_server.py`: advertise service, compute sum
- `add_two_ints_client.py`: call service, print result

What you should practice:
- defining callbacks for a service server
- waiting for a service and making an async call from the client

## 6.3. Example C — Fibonacci action server & client (actions)
**Goal**: implement a long-running goal with feedback.
- `fibonacci_action_server.py`
- `fibonacci_action_client.py`

What you should practice:
- accepting goals, publishing feedback, returning results
- cancelation handling (optional extension)


# 7. References + Additional Resources
- [ROS 2 Documentation](https://docs.ros.org/en/jazzy/index.html)
- [ROS Index](https://index.ros.org/)

# 8. Assignments
- Quiz 3: Homogeneous Transformations (Due Feb. 15, 11:59 pm)
- Quiz 4: ROS Fundamentals (Due Feb. 17, 11:59 pm)
- Assignment 2: ROS Message-passing (Due Feb. 17, 11:59 pm)